In [ ]:
# if the files are on Anvil / Terra workspace 
# please run it from the workspace analysis to avoid bucket access permission issues
# or use the gcloud login auth interface to authenticate yourself

In [ ]:
!pwd

installations:

1. pip install synapseclient

In [ ]:
!pwd

In [ ]:
import os
import pandas as pd

full_fragments_and_cell_type_labels = [("/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/fragment_files/syn52118181_syn52120036_ENCSR618WVK.atac.filter.fragments.hg38.tsv.gz/ENCSR618WVK.atac.filter.fragments.hg38.tsv.gz",
                                        "/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/labels/syn34271785_ENCSR618WVK_syn34545411/cell_types.tsv.gz")]
# full_fragments_and_cell_type_labels = [("/Users/eilaarich-landkof-stanford/Downloads/k562/k562_err9847049_rna_atac_common_filtered_fragments.tsv.gz",
#                                         "/Users/eilaarich-landkof-stanford/Downloads/k562/barcode_cell_types.tsv.gz")]



local_clusters_fld = os.path.join(os.getcwd(),"clusters")
os.makedirs(local_clusters_fld, exist_ok=True)
local_path_to_download = os.path.join(os.getcwd(),"fragment_files")

In [ ]:
local_clusters_fld

In [ ]:
local_path_to_download

In [ ]:
def remove_file(filename):
  print("remove_file method: {}".format(filename))
  if os.path.exists(filename):
    os.remove(filename)

In [ ]:
def save_clusters_mapping(fld,atac_dataset_id,list_to_save):
 # Create a DataFrame from the tuple list
    df = pd.DataFrame(list_to_save, columns=["ClusterId", "ManualAnnotationLabel"])

    # Specify the output TSV file path
    output_tsv_file = os.path.join(fld,atac_dataset_id,"{}_cluster_id_annotation_map.tsv".format(atac_dataset_id))
    print("sort_list_with_indices_and_save: output_tsv_file is {}".format(output_tsv_file))
    # Write the DataFrame to a TSV file
    df.to_csv(output_tsv_file, sep="\t", index=False)

In [ ]:
# IMPORTNAT: we make sure to keep the same order if the cell_id and the cluster for the next execution
def sort_list_with_indices(input_list):
    sorted_list = sorted(input_list)
    indexed_sorted_list = [(i, item) for i, item in enumerate(sorted_list)]

    
    return indexed_sorted_list

In [ ]:
%run tsv_files_utils_from_analysis.ipynb
from itertools import islice


for local_file_tuple in full_fragments_and_cell_type_labels:

    # Dictionary to store the output file handles with names
    output_handles = {}
    local_fragment_file = local_file_tuple[0]
    full_cell_types_annotation_file_path = local_file_tuple[1]
    print("!!!!!local_fragment_file is {}".format(local_fragment_file))
    file_atac_dataset_id = local_fragment_file.split("_")[-1].split(".")[0]
    print("$$$$$$file_atac_dataset_id is {}".format(file_atac_dataset_id))
 
    # df_cell_types_for_atac_dataset = read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines(full_cell_types_annotation_file_path)
    # skip_rows = 1 - tanjin
    skip_rows=1
    df_cell_types_for_atac_dataset = read_tsv_gz_to_dataframe_skipping_empty_lines_and_rows(full_cell_types_annotation_file_path,skip_rows)
    print("df_cell_types_for_atac_dataset.head(2) is {}".format(df_cell_types_for_atac_dataset.head(2)))
    cell_type_id_names_for_atac_dataset = list(set(df_cell_types_for_atac_dataset['cell_type_id'].to_list()))
    sorted_idx_cell_type_id_names_for_atac_dataset = sort_list_with_indices(cell_type_id_names_for_atac_dataset)
    print("%%%%%%sorted_idx_cell_type_id_names_for_atac_dataset is {}".format(sorted_idx_cell_type_id_names_for_atac_dataset))
    print("^^^^^^^number of cell_type_id_names_for_atac_dataset is {}".format(len(sorted_idx_cell_type_id_names_for_atac_dataset)))

    cell_id_to_type_dict = dict(zip(df_cell_types_for_atac_dataset['cell_id'], df_cell_types_for_atac_dataset['cell_type_id']))
    print("\n".join([f"Cell ID: {cell_id}, Cell Type ID: {cell_type_id}" for cell_id, cell_type_id in islice(cell_id_to_type_dict.items(), 5)]))

    # List of output text files with corresponding names
    # [("output1.txt", "file_1"), ("output2.txt", "file_2"), ("output3.txt", "file_3")]
    output_tagAlign_files_with_names = [(os.path.join(local_clusters_fld,file_atac_dataset_id,"{}_Cluster{}.tsv.tagAlign".format(file_atac_dataset_id,item[0])),
                                    item[1]) for item in sorted_idx_cell_type_id_names_for_atac_dataset]
    print("@@@@output_tagAlign_files_with_names is {}".format(output_tagAlign_files_with_names))

    # this will make sure that we will not run the same tagAlign twice.
    tagAlign_exists = [os.path.exists(output_tagAlign_file[0]) for output_tagAlign_file in output_tagAlign_files_with_names]
    print("tagAlign_exists is {}".format(tagAlign_exists))
    if sum(tagAlign_exists) >0:
        print("output_tagAlign_files_with_names {} is at work or was already downloaded. continue".format(output_tagAlign_files_with_names))
        continue # either started by annother process or already was processed
    else:
        print("!!!output_tagAlign_files_with_names {}. open files".format(output_tagAlign_files_with_names))
        for tag_file_path, tag_file_cell_type_name in output_tagAlign_files_with_names:
            # print("tag_file_path is {}".format(tag_file_path))
            # print("os.path.dirname(tag_file_path) is {}".format(os.path.dirname(tag_file_path)))
            os.makedirs(os.path.dirname(tag_file_path), exist_ok=True)
            output_handles[tag_file_cell_type_name] = open(tag_file_path, "w")

    print("save the cluster to annotation mapping")
    save_clusters_mapping(local_clusters_fld,file_atac_dataset_id,sorted_idx_cell_type_id_names_for_atac_dataset)
    
    # print("!!!output_tagAlign_files_with_names {}".format(output_tagAlign_files_with_names))
    print("open local_fragment_file {}".format(local_fragment_file))
    with gzip.open(local_fragment_file, "rt") as infile:
        missing_bc = 0
        # Open the output files and store their handles in the list
        num_of_lines_written=0
        for line_number, line in enumerate(infile, start=1):
            # # debug
            # if line_number > 10:
            #     continue 

            # here cases where bc_datasetId or datasetId_bc are being mixed between the fragments and the
            # cell type are being address. you can select the righ out_list for your experiment
            print("line is {}".format(line))
            out_list = split_fragment_line_string(line)
            print("out_list is {}".format(out_list))
            bc = out_list[-1]
            print("bc is {}".format(bc))
#             Austin output: chrom, start, end, bc, rem = line.rstrip('\n').split('\t', 5)
            out_line_to_print = f"{out_list[0]}\t{out_list[1]}\t{out_list[2]}\t{bc}\t{out_list[-2]}\n"
#             chr1	10007	10175	ENCSR023FME#ENCSR023FME_GAAGGTTCAAAGTGTCAGTCAA	1
            num_of_lines_written +=1
            returnTagAlign = convert_fragment_line_to_tagAlign(out_line_to_print)
            
            cell_id_to_type_dict[bc]
            try:
                tag_file_cell_type_id = cell_id_to_type_dict[bc] #df_cell_types_for_atac_dataset.loc[df_cell_types_for_atac_dataset['cell_id'] == bc, 'cell_type_id'].iloc[0]
                # print("tag_file_cell_type_id is {}".format(tag_file_cell_type_id))
                output_handles[tag_file_cell_type_id].write(returnTagAlign)
            except:
                missing_bc+=1
        
        for tag_file_path, tag_file_cell_type_name in output_tagAlign_files_with_names:
            print("tag_file_path is {}".format(tag_file_path))
            output_handles[tag_file_cell_type_name].close()    
        print("finished clustering local_fragment_file {} by cell type. for types {}".format(local_fragment_file, cell_type_id_names_for_atac_dataset))
        print("total missing bc are {}".format(missing_bc))

In [ ]:
# cell_id_to_type_dict['ENCSR618WVK-1_AAAGGCTCACCAGGTT']

In [ ]:
# df_cell_types_for_atac_dataset.head(2)

In [ ]:
# output_tagAlign_files_with_names

In [ ]:
# sorted_idx_cell_type_id_names_for_atac_dataset

In [ ]:
# !ls /Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters/


# df_cell_types_for_atac_dataset

In [ ]:
# cell_id_list = df_cell_types_for_atac_dataset['cell_id'].tolist()
# cell_type_id_list = df_cell_types_for_atac_dataset['cell_type_id'].tolist()

# print("Cell IDs List:", cell_id_list[0:4])
# print("Cell Type IDs List:", cell_type_id_list[0:4])


In [ ]:
# cell_id_to_type_dict = dict(zip(df_cell_types_for_atac_dataset['cell_id'], df_cell_types_for_atac_dataset['cell_type_id']))
# # cell_id_to_type_dict

In [ ]:
# import pandas as pd
# from itertools import islice

# # Assuming your DataFrame is named df_cell_types_for_atac_dataset
# # Replace df_cell_types_for_atac_dataset with the actual name of your DataFrame

# cell_id_to_type_dict = dict(zip(df_cell_types_for_atac_dataset['cell_id'], df_cell_types_for_atac_dataset['cell_type_id']))

# # Print the first 5 entries from the dictionary
# num_entries_to_print = 5
# print("\n".join([f"Cell ID: {cell_id}, Cell Type ID: {cell_type_id}" for cell_id, cell_type_id in islice(cell_id_to_type_dict.items(), num_entries_to_print)]))


In [ ]:
# cell_id_to_type_dict['GCCTGAdddddGGCTATGT']